In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_train = pd.read_csv('../data/train_cleaned.csv', index_col=0)
budget_test = pd.read_csv('../data/test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
for col in budget_test[word_predictors]:
    budget_test[col] = budget_test[word_predictors].loc[:,[col]].fillna(' ')

for col in budget_train[word_predictors]:
    budget_train[col] = budget_train[word_predictors].loc[:,[col]].fillna(' ')

In [4]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    Parameters
     ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    

# feature union transformer list 
transformer_list = [
    ('Object_Description', Pipeline([
                ('selector', ItemSelector(key='Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_2', Pipeline([
                ('selector', ItemSelector(key='Text_2')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('SubFund_Description', Pipeline([
                ('selector', ItemSelector(key='SubFund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Job_Title_Description', Pipeline([
                ('selector', ItemSelector(key='Job_Title_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_3', Pipeline([
                ('selector', ItemSelector(key='Text_3')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_4', Pipeline([
                ('selector', ItemSelector(key='Text_4')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Sub_Object_Description', Pipeline([
                ('selector', ItemSelector(key='Sub_Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Location_Description', Pipeline([
                ('selector', ItemSelector(key='Location_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Function_Description', Pipeline([
                ('selector', ItemSelector(key='Function_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Facility_or_Department', Pipeline([
                ('selector', ItemSelector(key='Facility_or_Department')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Position_Extra', Pipeline([
                ('selector', ItemSelector(key='Position_Extra')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Program_Description', Pipeline([
                ('selector', ItemSelector(key='Program_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Fund_Description', Pipeline([
                ('selector', ItemSelector(key='Fund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_1', Pipeline([
                ('selector', ItemSelector(key='Text_1')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
]

In [5]:
label = LabelEncoder()
X_test = budget_test

#create sub data sets for blendedmodels
X_train = budget_train[word_predictors] 
y_train = label.fit_transform(budget_train['Reporting'])
X_sub_1, X_sub_2, y_sub_1, y_sub_2 = train_test_split(X_train,y_train, test_size = .5, random_state = 42)
X_test = budget_test

#first layer models
lr_pipe_1 = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression())])

lr_pipe_2 = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression(C=10))])

#fit models on first subset 
lr_pipe_1.fit(X_sub_1,y_sub_1)
lr_pipe_2.fit(X_sub_1,y_sub_1)
   
#predict proba on second subset of data and blend the results together 
lr1_train_predict = lr_pipe_1.predict_proba(X_sub_2)
lr2_train_predict = lr_pipe_2.predict_proba(X_sub_2)
blended = np.concatenate([lr1_train_predict, lr2_train_predict], axis=1)


# get predict_proba test_set and combine results together 
lr1_test_predict = lr_pipe_1.predict_proba(X_test)
lr2_test_predict = lr_pipe_2.predict_proba(X_test)
test_combine_pred = np.concatenate([lr1_test_predict, lr2_test_predict], axis=1)

#fit blended predict_probas and y_sub_2 in blender model
blender = RandomForestClassifier()
blender.fit(blended, y_sub_2)

# get predict proba of the blended result from blender model
train_prediction = blender.predict_proba(blended)

log_loss_score = log_loss(y_sub_2, train_prediction)

# get predict_proba of blended test set from
test_blended_proba = blender.predict_proba(test_combine_pred)


In [9]:
train_prediction = blender.predict_proba(blended)
Reporting_result = [{"target": "Reporting",
             "log_loss": log_loss(y_sub_2, train_prediction)
             }]
Reporting_result = pd.DataFrame(Reporting_result)

# Reporting_result.to_csv('Reporting_blended.csv')

#find predict proba with blended model of blended test_proba    
blended_proba = blender.predict_proba(test_combine_pred)

Reporting = pd.DataFrame(blended_proba,columns=label.classes_)

In [10]:
Reporting_result

,log_loss,target
0,0.034496,Reporting


In [12]:
Reporting.to_pickle('Reporting.pkl')